In [1]:
###IMPORT MODULES
import sys
import os
sys.path.append(os.path.abspath('../..'))

import numpy as np 
from tools.component_tools import Component
from tools.component_tools import Fluid
from tools.component_tools import Membrane
import tools.materials as materials
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from tools.component_tools import BreedingBlanket
import mph 
from tools.component_tools import Geometry


U0=2
eff_coms_v=np.array([])
eff_py=np.array([])
client = mph.start()
compute_comsol=True
# U0=1.5
T=800
c0=1E-2
mat=materials.Flibe(T)
T_hot_prim=900
T_cold_prim=800
Q=1E9
d_hyd=1E-2
L=20
D=1E-4
thick=2E-3
K_S=2E-2
k_d=1E6
k_r=1E6
K_H=1E-1
flibe=Fluid(T=T, Solubility=K_H, MS=True,D=mat.D, d_Hyd=d_hyd ,mu=mat.mu,rho=mat.rho,U0=U0,k=mat.k,
        cp=mat.cp)
var_vec=[T,c0,d_hyd,L,D,thick,K_S,K_H,mat.D,U0]
var_vec_str=['T','c0','d_hyd','L','D_St','thick','K_S','K_H','D_flibe','U0']
Steel = Membrane( T=T,
        D=D,
        thick=thick,
        K_S=K_S,
        k_d=k_d,
        k_r=k_r,k=21)
geom_pav=Geometry(L=L, D=d_hyd,thick=thick )
PAV = Component(c_in=c0,  fluid=flibe, membrane=Steel,geometry=geom_pav)
PAV.get_solid_inventory()
PAV.use_analytical_efficiency()
print("Efficiency "+str(PAV.eff))
if compute_comsol:
        client = mph.start()
        if PAV.fluid.MS: 
                
                model = client.load('PAV_MS_2species_SST_copia.mph')  
        else: 
                model = client.load('PAV_LM_SST.mph')
        for i,var in enumerate(var_vec):
                model.parameter(var_vec_str[i], str(var))
        model.mesh()
        a=model.parameters()
        # model.save()

        model.solve()
        # Evaluate the results
        eff_coms = model.evaluate(['r', 'z', 'inv'], 'Study 1//Solution 1')

        # Remove NaN values from the 'inv' component
        inv_values = eff_coms[-1]  # Assuming 'inv' is the last component
        inv_values = inv_values[~np.isnan(inv_values)]

        # Calculate the average of the cleaned 'inv' values
        eff_coms_avg = np.average(inv_values)
        eff_coms_v = np.append(eff_coms_v, eff_coms_avg)
else: 
        eff_coms_avg=4.22E-6
        # Append the average to the eff_coms_v array


# Print the results
print("PAV", (PAV.membrane.inv), "Coms=", eff_coms_avg, "%", "diff=", PAV.membrane.inv - eff_coms_avg, "%", "rel_diff=", (abs(PAV.membrane.inv - eff_coms_avg) / PAV.membrane.inv) * 100, "%")
if PAV.fluid.MS and compute_comsol:
        k_t_coms=model.evaluate(['r', 'z', 'k_t'], 'Study 1//Solution 1')
        k_t_values=k_t_coms[-1]
        k_t_values=k_t_values[~np.isnan(k_t_values)]
        k_t_avg=np.average(k_t_values)
        PAV.fluid.k_t=k_t_avg
        PAV.get_solid_inventory()
        print("PAV", (PAV.membrane.inv), "Coms=", eff_coms_avg, "%", "diff=", PAV.membrane.inv - eff_coms_avg, "%", "rel_diff=", (abs(PAV.membrane.inv - eff_coms_avg) / PAV.membrane.inv) * 100, "%")

        # Append the PAV membrane inventory to the eff_py array


alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.009158361191324281+0j)
alpha457.09130351857027corr(0.0091583611913242

C:\Users\Gabriele\AppData\Roaming\Python\Python311\site-packages\scipy\integrate\_quadpack_py.py:577: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


PAV 9.088243336675977e-09 Coms= 5.1416740957496666e-09 % diff= 3.946569240926311e-09 % rel_diff= 43.424995290341414 %
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.009526021077177442+0j)
alpha1498.4384835199367corr(0.0095

In [2]:

from sympy import comp


print("ratio", PAV.membrane.inv/eff_coms_avg)

# Remove the model from the client
if compute_comsol:
    client.remove(model)


ratio 1.0145280892660438
